In [2]:
import pandas as pd

In [3]:
dbo = pd.read_excel('dbo.events_3.xlsx')

Описание задачи

Банк разместил рекламу в Яндекс Директе в рамках маркетинговой акции "бесплатное обслуживание счёта в первый год"

Есть база данных:

event_datetime: время события, только 2020 год

event_id: идентификатор события, первичный ключ

user_id: идентификатор посетителя (один пользователь может несколько раз заходить на сайт, в том числе и в течение дня, в том числе с разных городов). Так же посещения могут быть уже после открытия счета

event_name: название события (Посещение сайта или Открытие счета). Для упрощения - счет может открыться только после посещения сайта, и каждый посетитель может открыть только один счет

city: город

cost: расчетная стоимость привлечения посетителя на сайт

In [5]:
dbo.event_datetime = pd.to_datetime(dbo.event_datetime)

In [7]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
dbo.to_sql('dbo', con=engine, index=False)

29

In [8]:
sql_code = """
            

                    SELECT *
                    FROM dbo


        """
temp = pd.read_sql(sql_code, engine)
temp

,event_datetime,event_id,user_id,event_name,city,cost
0,2020-01-05 05:40:00.000000,31130,17,Посещение сайта,Зеленодольск,0.92
1,2020-01-05 05:40:00.000000,31131,10,Посещение сайта,Пермь,5.91
2,2020-01-05 05:40:00.000000,31132,63,Посещение сайта,Казань,7.34
3,2020-02-05 05:40:00.000000,31133,17,Открытие счета,Череповец,5.05
4,2020-01-05 05:41:00.000000,31134,70,Посещение сайта,Пермь,7.31
5,2020-01-05 05:41:00.000000,31135,84,Посещение сайта,Казань,NaN
6,2020-01-05 05:41:00.000000,31136,84,Посещение сайта,Череповец,0.75
7,2020-01-05 05:41:00.000000,31137,31,Посещение сайта,Пермь,0.78
8,2020-09-05 05:41:00.000000,31138,84,Открытие счета,Казань,NaN
9,2020-01-05 05:41:00.000000,31139,52,Посещение сайта,Казань,7.14


1. Выведем количество уникальных посетителей, которые заходили на сайт из разных городов

In [9]:
sql_code = """
            

                    SELECT city, 
                           COUNT(DISTINCT user_id) AS num_user_id
                    FROM dbo
                    GROUP BY city


        """
temp = pd.read_sql(sql_code, engine)
temp

,city,num_user_id
0,Зеленодольск,2
1,Казань,8
2,Пермь,6
3,Череповец,6


2. Найдем город, с максимальным числом уникальных посетителей. Выведем в разрезе каждого месяца

In [11]:
sql_code = """
            

                    WITH t AS
                    (
                        SELECT STRFTIME('%m', event_datetime) AS month, 
                               city, 
                               COUNT(DISTINCT user_id) AS num_user_id
                        FROM dbo
                        GROUP BY month, city
                        ORDER BY COUNT(DISTINCT user_id) DESC
                    )
            
                    SELECT month, city
                    FROM
                    (
                        SELECT *,
                               ROW_NUMBER() OVER(PARTITION BY month) AS rownum
                        FROM t
                    )
                    WHERE rownum = 1


        """
temp = pd.read_sql(sql_code, engine)
temp

,month,city
0,01,Казань
1,02,Зеленодольск
2,03,Череповец
3,04,Казань
4,08,Череповец
5,09,Казань
6,11,Череповец


3. Найдем среднее количество посещений сайта, потребовавшееся посетителям, чтобы принять решение об открытии счета ( результат = одно число, только по посетителям, открывшим счет)

Если пойти простым путём и предположить, что дата открытия р/с для каждого конкрентного id всегда больше даты посещения сайта, например для этого визитёра потом меняется id на логин и производится дальшейшее фиксирование его посещений уже как аутентифицированного пользователя и т.п., то делаем так:

In [12]:
sql_code = """
            

                    SELECT AVG(num_visits)
                    FROM
                    (
                        SELECT user_id, COUNT(*) AS num_visits
                        FROM dbo
                        WHERE user_id IN
                        (
                            SELECT DISTINCT user_id
                            FROM dbo
                            WHERE event_name = 'Открытие счета'
                        )
                        AND event_name = 'Посещение сайта'
                        GROUP BY user_id
                    )

  
        """
temp = pd.read_sql(sql_code, engine)
temp

,AVG(num_visits)
0,2.0


Важно: ивент с открытием р/с не принимаем за одно из нужных нам посещений, а считаем только визиты ДО открытия - если он пришёл на сайт и сразу открыл счёт - решение он принял до этого)

Если мы также продолжаем фиксировать те же id  с посещениями и после открытия р/с надо их исключать, ведь решение он уже принял и счёт открыл, мб тарифы решил глянуть?

In [13]:
sql_code = """
            

                    WITH t AS
                    (
                        SELECT *
                        FROM dbo
                        LEFT JOIN
                        (
                            SELECT user_id, 
                                   event_datetime AS open_datetime
                            FROM dbo
                            WHERE event_name = 'Открытие счета'
                        ) AS dbo_open
                        ON dbo.user_id = dbo_open.user_id
                        WHERE event_datetime < open_datetime
                    )
                    
                    SELECT AVG(num_visits) AS avg_visits
                    FROM
                    (
                        SELECT user_id, COUNT(*) AS num_visits
                        FROM t
                        GROUP BY user_id
                    )

  
        """
temp = pd.read_sql(sql_code, engine)
temp

,avg_visits
0,1.8


4. Найдем среднее число дней , потребовавшееся посетителям, чтобы принять решение об открытии счета   ( Результат = одно число, только по посетителям, открывшим счет, считаем с первого посещения сайта)

In [14]:
sql_code = """
            

                    WITH t AS
                    (
                        SELECT *
                        FROM dbo
                        LEFT JOIN
                        (
                            SELECT user_id, 
                                   event_datetime AS open_datetime
                            FROM dbo
                            WHERE event_name = 'Открытие счета'
                        ) AS dbo_open
                        ON dbo.user_id = dbo_open.user_id
                        WHERE event_datetime < open_datetime
                    )
                    
                    SELECT AVG(delta) AS avg_days
                    FROM
                    (
                        SELECT user_id, 
                               MAX(delta) AS delta
                        FROM
                        (
                            SELECT event_datetime, 
                                   user_id, 
                                   (JULIANDAY(open_datetime) - JULIANDAY(event_datetime)) AS delta
                            FROM t
                        )
                        GROUP BY user_id
                    )
              
  
        """
temp = pd.read_sql(sql_code, engine)
temp

,avg_days
0,170.600278


5. Выведем идентификаторы посетителей, которые открыли счет в день первого посещения сайта

In [15]:
sql_code = """
            
                    
                    WITH t AS
                    (
                        SELECT user_id, 
                               MIN(event_datetime) AS first_visit
                        FROM
                        (
                            SELECT user_id, 
                                   STRFTIME('%Y-%m-%d', event_datetime) AS event_datetime
                            FROM dbo
                            WHERE event_name != 'Открытие счета'
                        )
                        GROUP BY user_id
                    )
                    
                    SELECT DISTINCT user_id
                    FROM
                    (
                        SELECT *
                        FROM t
                        JOIN
                        (
                            SELECT user_id,
                                   STRFTIME('%Y-%m-%d', event_datetime) AS open_datetime
                            FROM dbo
                            WHERE event_name = 'Открытие счета'
                        ) AS dbo_open
                        ON t.user_id = dbo_open.user_id
                    )
                    WHERE first_visit = open_datetime


        """
temp = pd.read_sql(sql_code, engine)
temp

,user_id
0,77


6. Найдем общую стоимость привлечения и количество посетителей, которые в течение года так и не  открыли счет 

In [16]:
sql_code = """


                    SELECT COUNT(DISTINCT user_id) AS no_open_visiters, 
                           SUM(cost) AS no_open_cost
                    FROM dbo
                    WHERE user_id NOT IN
                    (
                        SELECT DISTINCT user_id
                        FROM dbo
                        WHERE event_name = 'Открытие счета'
                    )

  
        """
temp = pd.read_sql(sql_code, engine)
temp

,no_open_visiters,no_open_cost
0,7,48.88


7. Найдем общую стоимость затрат на посещения после открытия счета (впустую потраченных денег, с учетом того, что счет можно открыть только один раз)

In [17]:
sql_code = """


                    SELECT SUM(cost) AS lost_money
                    FROM dbo
                    LEFT JOIN
                    (
                        SELECT user_id, 
                               event_datetime AS open_datetime
                        FROM dbo
                        WHERE event_name = 'Открытие счета'
                    ) AS dbo_open
                    ON dbo.user_id = dbo_open.user_id
                    WHERE event_datetime > open_datetime


        """
temp = pd.read_sql(sql_code, engine)
temp

,lost_money
0,5.05


8. Найдем общую стоимость привлечения и количество посетителей, которые в течение года открыли счет 

ТУТ МОЖНО ПОСЧИТАТЬ А) ОБЩАЯ СТОИМОСТЬ ПО ID, КОТОРЫЕ ОТКРЫЛИ Р/С Б) СТОИМОСТЬ ПО ТЕМ ИВЭНТАМ, КОТОРЫЕ БЫЛИ до ДАТЫ ОТКРЫТИ Р/С

А

In [18]:
sql_code = """            
                    

                    SELECT COUNT(DISTINCT user_id) AS open_visiters, 
                           SUM(cost) AS open_cost
                    FROM dbo
                    WHERE user_id IN
                    (
                        SELECT DISTINCT user_id
                        FROM dbo
                        WHERE event_name = 'Открытие счета'
                    )


        """
temp = pd.read_sql(sql_code, engine)
temp

,open_visiters,open_cost
0,6,50.41


Б

In [19]:
sql_code = """            


                    SELECT COUNT(DISTINCT dbo.user_id) AS open_visiters, 
                           SUM(cost) AS open_cost
                    FROM dbo
                    LEFT JOIN
                    (
                        SELECT user_id, 
                               event_datetime AS open_datetime
                        FROM dbo
                        WHERE event_name = 'Открытие счета'
                    ) AS dbo_open
                    ON dbo.user_id = dbo_open.user_id
                    WHERE event_datetime <= open_datetime

  
        """
temp = pd.read_sql(sql_code, engine)
temp

,open_visiters,open_cost
0,6,45.36


9. Выведем уникальные даты посещения сайта посетителем, стоимость привлечения которого на сайт (Событие "Посещение сайта", за весь год) оказалась самой высокой. Независимо от того, открыл или не открыл счет

In [20]:
sql_code = """            


                    SELECT DISTINCT STRFTIME('%Y-%m-%d', event_datetime) AS event_datetime
                    FROM dbo
                    WHERE user_id IN
                    (
                        SELECT user_id
                        FROM dbo
                        GROUP BY user_id
                        ORDER BY SUM(cost) DESC
                        LIMIT 1
                    )

  
        """
temp = pd.read_sql(sql_code, engine)
temp

,event_datetime
0,2020-01-05


10.Найдем ежемесячное изменение количества уникальных посетителей сайта (в процентах) . Результат должен быть примерно таким январь (или 1) - NULL, февраль (или 2) - "-5%" (по отношению к январю), март (или 3) - "9%" (по отношению к февралю)

In [92]:
sql_code = """            


                    WITH t AS
                    (
                        SELECT STRFTIME('%m', event_datetime) AS event_month, 
                               COUNT(DISTINCT user_id) AS num_visiters
                        FROM dbo
                        WHERE event_name = 'Посещение сайта'
                        GROUP BY STRFTIME('%m', event_datetime)
                    )


                    SELECT event_month, 
                           (ROUND((CAST(num_visiters AS FLOAT)/CAST(last_month AS FLOAT))*100, 2)-100) || '%' AS percent
                    FROM
                    (
                        SELECT *, 
                        LAG(num_visiters, 1, 0) OVER(ORDER BY event_month) AS last_month
                        FROM t
                    )
         
                        
        """
temp = pd.read_sql(sql_code, engine)
temp

,event_month,percent
0,01,None
1,02,-70.0%
2,03,0.0%
3,04,-66.67%
